# Sea ice prediction notebook

Date: 09/12/2017  
Author: Alek Petty  

Description: Notebook for sea ice prediction experimentation  
Based on the get_multivar_forcast_months_exp.py script.

Notes: 
This only generates forecasts for one year (of your choosing). The Python script is set up as a loop calling a function to produce forecasts for a series of years. This is needed to produce the lineplots of all the different forecasts and to produce an assessment of forecast skill. This Notebook is more to demonstrate and experiment with the forecast model


In [1]:
# Import Python libraries
%matplotlib inline

import argparse # for passing variables from bash
import sys
import forecast_funcs as ff
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from pylab import *

/Applications/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/Applications/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Options needed to run this Script

plotSkill=1 # for plotting the skill
outSkill=1 # far saving the skill values
outLine=1 # for saving the forecast time series
outWeights=1 #for saving the weightings

startYearF=1979 # Start year of forecast training
numYearsReq=5 #number of years required in a grid cell for it to count towards the training data

yearF=2012 # Forecast data year
yearP=2012 # Predicted year

fmonth=6 #6=June, 9=Sep #  Forecast month
pmonth=9 #9=SEP # Predicted month
fvars=['conc'] # Sea ice concentration='conc'. 
concVersion='v2' #version of the gridded ice conc files
# Can add multiple variables here, e.g. 'melt'= melt onset over sea ice
weight=1 # Spatially weighting the data, normally 1
hemStr='N' # Hemipshere (N or S)
iceType='extent' # ice type being forecast ('extent' or 'area')
alg=0

region=0
#0 implies pan-Arctic or Antarctic
#2 Weddell Sea
#3 Indian Ocean
#4 Pacific Ocean
#5 Ross Sea
#6 Amundsen/BHausen Sea
#A Alaskan

anomObs=1 # This flag means we have observed data for the forecast year.
# This needs to be set to zero if producing forecasts into the future.


In [3]:
# Figure paths
figpath='../Figures/'
rawdatapath='../../../../DATA/'
extdatapath = '../DataOutput/Extent/'
dataoutpath='../DataOutput/'

if (hemStr=='S'):
	skilldatapath='../DataOutput/Antarctic/SkillVals/'
	linedatapath='../DataOutput/Antarctic/TimeSeries/'
	datapath='../DataOutput/Antarctic/'
elif (hemStr=='N'):
	skilldatapath='../DataOutput/Arctic/SkillVals/'
	linedatapath='../DataOutput/Arctic/TimeSeries/'
	datapath='../DataOutput/Arctic/'

if (region=='A'):
	skilldatapath='../DataOutput/Alaska/SkillVals/'
	linedatapath='../DataOutput/Alaska/TimeSeries/'

In [4]:
# Get the training data for all years prior to the given forecast year
yearsTr, extentTr = ff.getIceExtentAreaPetty(extdatapath, pmonth, startYearF, yearP-1, icetype=iceType, alg=alg)

# Linearly detrend the data
# NB THIS IS SOMETHING WE CAN LOOK INTO CHANGING
extentDTr, lineTr=ff.get_varDT(yearsTr, extentTr)
print extentTr, extentDTr

[ 7.26032299  7.88943296  7.28314957  7.48270218  7.57175158  7.13994098
  6.96138861  7.5864269   7.54152359  7.53465491  7.08755243  6.27644633
  6.59263747  7.5860196   6.54160085  7.24209901  6.18037068  7.9096321
  6.77434702  6.61537345  6.28588959  6.35572741  6.77162492  5.97588484
  6.17904505  6.07441663  5.59171572  5.94600097  4.32049685  4.74593519
  5.39666295  4.94279883  4.64385147] [ -6.51174733e-01   6.27685686e-02  -4.58681484e-01  -1.74295535e-01
  -4.12804805e-04  -3.47390072e-01  -4.41109102e-01   2.68762529e-01
   3.08692547e-01   3.86657202e-01   2.43880583e-02  -7.01884702e-01
  -3.00860229e-01   7.77355241e-01  -1.82230172e-01   6.03101324e-01
  -3.73793676e-01   1.44030108e+00   3.89849341e-01   3.15709107e-01
   7.10585829e-02   2.25729741e-01   7.26460579e-01   1.55538384e-02
   3.03547386e-01   2.83752300e-01  -1.14115273e-01   3.25003314e-01
  -1.21566747e+00  -7.05395793e-01   3.01653024e-02  -3.38865486e-01
  -5.52979510e-01]


In [5]:
# Generate the forecast model!
# THIS IS THE MAIN PART OF THE MODEL THAT CAN LIKELY BE IMPROVED.

# Need to get an array filled with ones to act as the intercept
predVarsTYr=[1]
predVars=np.ones((size(yearsTr)))

# Get forecast years
# Normally same as yearsTr but in case crossing year I added this to main script
#yearsFr=np.arange(startYearF, yearF, 1)

for varT in fvars:
    #print 'Var:', varT
    if (varT in ['sst','conc','melt','melt_nan', 'pmas']):

        # Get the gridded forecast data for training (prior to forecast year)
        VarYearsTr = ff.get_gridvar(varT, fmonth, yearsTr, hemStr)

        # Get the gridded data of the forecast year
        VarYear = ff.get_gridvar(varT, fmonth, array(yearF), hemStr)

        # Weight the gridded forecast data with training/historical sea ice data
        rvalsDT, unweightedPredVarT, predVarT, predVarTYr = ff.GetWeightedPredVar(yearsTr, yearF, extentDTr, VarYearsTr, VarYear,varT, fmonth, pmonth, startYearF,numYearsReq, region, hemStr, iceType, normalize=0, outWeights=outWeights, weight=weight)

    # will be an array of 1 (intercept) and a number
    predVarsTYr.append(predVarTYr)
    # will be an array of 1s (intercepts) and a series of numbers
    predVars=np.column_stack((predVars, array(predVarT)))


In [6]:
predVars

array([[  1.00000000e+00,  -1.11533558e-03],
       [  1.00000000e+00,  -1.57141479e-03],
       [  1.00000000e+00,  -2.48682427e-03],
       [  1.00000000e+00,   3.63248996e-04],
       [  1.00000000e+00,  -9.02611424e-05],
       [  1.00000000e+00,   4.59026913e-04],
       [  1.00000000e+00,  -1.82386540e-03],
       [  1.00000000e+00,   4.54861550e-04],
       [  1.00000000e+00,   9.92024856e-04],
       [  1.00000000e+00,  -5.66744090e-05],
       [  1.00000000e+00,   2.40897113e-03],
       [  1.00000000e+00,  -3.53930532e-03],
       [  1.00000000e+00,  -2.94340521e-05],
       [  1.00000000e+00,   2.81033874e-03],
       [  1.00000000e+00,  -2.56679292e-03],
       [  1.00000000e+00,   2.75260177e-03],
       [  1.00000000e+00,  -2.91926487e-03],
       [  1.00000000e+00,   5.14470090e-03],
       [  1.00000000e+00,   6.03178659e-04],
       [  1.00000000e+00,   2.66052118e-04],
       [  1.00000000e+00,   2.11958169e-03],
       [  1.00000000e+00,   3.11220300e-04],
       [  

In [7]:
# Use SM to generate the regression model. Could have just used linregress (tested, gave same results, but this was just a bit neater)
model=sm.OLS(extentDTr, predVars)
fit=model.fit()

# Forecast detrended sea ice extent!
extentForrDT = fit.predict(predVarsTYr)[0]

# Prediction uncertainty estimate
prstd, iv_l, iv_u = wls_prediction_std(fit, exog=predVarsTYr)

# Calculate alternative ice extent forecast simply assuming linear trend persistnce
extTrendP=(lineTr[-1]+(lineTr[-1]-lineTr[-2]))

extentForrAbs = extentForrDT+extTrendP

if (anomObs==1):
    # Get the extent data for the given prediction year and compare against forecast
    yearsYr, extentYr = ff.getIceExtentAreaPetty(extdatapath, pmonth, startYearF, yearP, icetype=iceType, alg=0)
    extentObsDT=extentYr-extentYr[-1]
    anom=extentYr-extentForrAbs
    print 'Observed extent:',extentYr[-1]

print 'Linear trend extent:',extTrendP
print 'Forecast extent:',extentForrAbs


Observed extent: 3.640041754
Linear trend extent: 5.11199764301
Forecast extent: 4.1835532317


In [8]:
yearsFr=np.arange(startYearF, yearF, 1)
print yearsFr

[1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008
 2009 2010 2011]


In [9]:
print predVarTYr.shape

()


In [10]:
# plot forecast anomaly field that drove the forecast

from mpl_toolkits.basemap import Basemap, shiftgrid
# plotting parameters
rcParams['xtick.major.size'] = 2
rcParams['ytick.major.size'] = 2
rcParams['axes.linewidth'] = .3
rcParams['lines.linewidth'] = .3
rcParams['patch.linewidth'] = .3

rcParams['axes.labelsize'] = 14
rcParams['xtick.labelsize']=14
rcParams['ytick.labelsize']=14
rcParams['legend.fontsize']=14
rcParams['font.size']=14
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})


minval=-0.5
maxval=0.5

if (hemStr=='N'):
    if ((region=='A')):
        regionOut='Alaska'
    else:
        regionOut='Arctic'
else:
    regionOut='Antarctic'

# Map projection
m = Basemap(projection='npstere',boundinglat=65,lon_0=0, resolution='l'  )

# Gridded coordinates

hemStr='N'
if (hemStr=='S'):
	poleStr='AA'
elif (hemStr=='N'):
	poleStr='A'

xpts=load(dataoutpath+'IceConcA/'+concVersion+'/xpts100km'+poleStr)
ypts=load(dataoutpath+'IceConcA/'+concVersion+'/ypts100km'+poleStr)
fig = figure(figsize=(9,9*0.8))
ax=gca()

im1 = m.contourf(xpts , ypts, rvalsDT,levels=[0.3, 0.8], colors='none', hatches=['//'], zorder=3)

im2 = m.pcolormesh(xpts, ypts, unweightedPredVarT,vmin=minval, vmax=maxval, cmap=cm.RdBu_r, shading='interp', zorder=2)
#im1 = m.pcolormesh(xpts , ypts, rvals, cmap=cm.cubehelix, vmin=minval, vmax=maxval,shading='flat', zorder=2)

m.fillcontinents(color='w',lake_color='0.9', zorder=2)
m.drawcoastlines(linewidth=0.25, zorder=5)
m.drawparallels(np.arange(90,-90,-10), linewidth = 0.25, zorder=3)
m.drawmeridians(np.arange(-180.,180.,30.), linewidth = 0.25, zorder=3)

label_str=r'$\Delta$ A'
cax = fig.add_axes([0.81, 0.05, 0.03, 0.45])
cbar = colorbar(im2,cax=cax, orientation='vertical', extend='both', use_gridspec=True)
cbar.set_label(label_str, labelpad=4, rotation=0)
cbar.set_ticks(np.linspace(minval, maxval, 3))
cbar.solids.set_rasterized(True)

#ax.annotate(varStr, xy=(0.5, 1.01),xycoords='axes fraction', horizontalalignment='center', verticalalignment='bottom', zorder=10)
ax.annotate(regionOut+'\n'+str(yearP)+'\nFM:'+str(fmonth)+' PM:'+str(pmonth)+'\n\nFvar:'+fvars[0]+'\nPvar:'+iceType, xy=(1.005, 0.98), xycoords='axes fraction', horizontalalignment='left', verticalalignment='top', rotation=0, zorder=10)

subplots_adjust(bottom=0.01, top=0.99, left=0.01, right=0.8)
plt.show()
#savefig(figpath+'anomaly'+varStr+iceType+'fm'+str(fmonth)+'pm'+str(pmonth)+'R'+str(region)+str(startYear)+str(yearT)+poleStr+'.png', dpi=300)
#close(fig)


IOError: [Errno 2] No such file or directory: '../DataOutput/IceConcA/xpts100kmA'